In [248]:
# Load the data and libraries
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')

# Load data files
import numpy as np
import urllib.request
import io

url_x = 'https://github.com/jnear/cs211-data-privacy/raw/master/slides/adult_processed_x.npy'
url_y = 'https://github.com/jnear/cs211-data-privacy/raw/master/slides/adult_processed_y.npy'

with urllib.request.urlopen(url_x) as url:
    f = io.BytesIO(url.read())
X = np.load(f)

with urllib.request.urlopen(url_y) as url:
    f = io.BytesIO(url.read())
y = np.load(f)

In [249]:
import torch

In [250]:
from timeit import default_timer as timer

In [251]:
def gaussian_mech_zCDP_vec(vec, sensitivity, rho):
    sigma = np.sqrt((sensitivity**2) / (2 * rho))
    return vec + np.random.normal(loc=0, scale=sigma, size=vec.shape)

In [252]:
# Split data into training and test sets
training_size = int(X.shape[0] * 0.8)

X_train = X[:training_size]
X_test = X[training_size:]

y_train = y[:training_size]
y_test = y[training_size:]

print('Train and test set sizes:', len(y_train), len(y_test))

Train and test set sizes: 36176 9044


# Clipping and Gradient definition

In [253]:
def L2_clip_array(vs , b):
    norms = np.linalg.norm(vs, ord = 2, axis = 1)
    ratios = vs/norms[:, None]
    results = np.where((norms > b)[:, None], b*ratios, vs)
    return results

In [254]:
def vgradient(theta_in, x_in, y_in, C):
    x = x_in
    y = y_in
    theta = theta_in
    exponent = y * np.dot(x, theta)
    rhs = (y/(1+np.exp(exponent)))
    gradients = -(x*rhs[:, None])
    clipped_grads = L2_clip_array(gradients, C)
    return np.sum(clipped_grads, axis = 0)

In [255]:
# Prediction: take a model (theta) and a single example (xi) and return its predicted label
def predict(xi, theta, bias=0):
    label = np.sign(xi @ theta + bias)
    return label

def accuracy(theta):
    return np.sum(predict(X_test, theta) == y_test)/X_test.shape[0]

# Baseline (gradient clipping DP-SGD)

In [256]:
def dp_gradient_descent(epochs, rho):
    rho_i = rho/epochs
    theta = np.zeros(X_train.shape[1])  # leaks the number of features, without privacy
    clipping_param = 1
    num_examples = X_train.shape[0]     # leaks the number of training examples, without privacy

    BATCH_SIZE = 256
    num_batches = int(num_examples / BATCH_SIZE)
    batches_X = np.array_split(X, num_batches)
    batches_y = np.array_split(y, num_batches)

    for i in range(epochs):

        for xs, ys in zip(batches_X, batches_y):
            grad_sum        = vgradient(theta, xs, ys, clipping_param)
            noisy_grad_sum  = gaussian_mech_zCDP_vec(grad_sum, clipping_param, rho_i)
            noisy_avg_grad  = noisy_grad_sum / BATCH_SIZE
            theta           = theta - noisy_avg_grad

    return theta

In [257]:
def zcdp_eps(rho, delta):
    return rho + 2*np.sqrt(rho * np.log(1/delta))
zcdp_eps(0.001, 1e-5)

0.21559660262893474

In [258]:
rho = .001
epochs = 5
print('eps:', zcdp_eps(rho, 1e-5))
accs = [accuracy(dp_gradient_descent(epochs, rho)) for _ in range(10)]
print('mean:', np.mean(accs))
print('std:', np.std(accs))

eps: 0.21559660262893474
mean: 0.7634453781512606
std: 0.026823566224831792


# Calvin's Sensitivity Implementation

In [453]:
def compute_sens_torch(many_xs, t, m, a, b, device):
    # Clamp and sort all xs's at once, assuming shape is (num_iter, n)
    many_xs = many_xs.clamp(a, b).sort(dim=1).values
    n = many_xs.size(1)
    assert n - 2*m > 0, f'm = {m} and n = {n} means n - 2m = {n-2*m} is negative'

    num_iter = many_xs.size(0)

    # Concat [b, a] to the end of every xs so that indexing -1 gives a and indexing n gives b, then clamp indices to -1, n
    many_xs = torch.cat((many_xs, torch.full((num_iter, 1), b, dtype=torch.float, device=device), 
                         torch.full((num_iter, 1), a, dtype=torch.float, device=device)), dim=1)

    # Generate indices now so they don't need to be every time (will be xs[idx1] - xs[idx2]), this doesn't need to be efficient but w/e
    ks = torch.arange(0, n+1, device=device) # distances
    #ks = torch.arange(0, 5, device=device) # distances
    ls = torch.arange(0, n+2, device=device)
    # Use all l values then take lower triangular part of matrix plus (with diagonal shifted by one) to remove values where l > k+1
    idx1 = torch.tril(n - m + 1 + ks.reshape(-1, 1) - ls, diagonal=1).clamp(-1, n)
    idx2 = (m + 1 - ls).clamp(-1, n)

    scalar = torch.exp(-1 * ks * t)

    out = torch.empty(num_iter)
    for i in range(num_iter):
        xs = many_xs[i]

        diffs = torch.tril(torch.abs(xs[idx1] - xs[idx2]), diagonal=1)
        inner_max = diffs.max(dim=1).values
        outer_max = (inner_max*scalar).max()
        out[i] = outer_max / (n - 2*m)

    return out


# Smooth Sensitivity DP-SGD

In [454]:
def lln(sigma, size):
    x = np.random.laplace(size=size)
    y = np.random.normal(size=size)
    return x * np.exp(sigma * y)

In [501]:
def vgradient_per_ex(theta, x, y):
    exponent = y * np.dot(x, theta)
    rhs = (y/(1+np.exp(exponent)))
    gradients = -(x*rhs[:, None])
    return gradients

def smooth_dp_gradient_descent(epochs, rho):
    rho_i = rho/epochs
    theta = np.zeros(X_train.shape[1])  # leaks the number of features, without privacy
    num_examples = X_train.shape[0]     # leaks the number of training examples, without privacy

    upper = 1
    lower = -1
    m = 10
    t = 1.0
    
    BATCH_SIZE = 256

    rho_weight = rho_i / X_train.shape[1]
    print('target per-weight rho:', rho_weight)

    sigma, s = optimize_sigma(rho_weight, t)    
    print('sigma:', sigma, 's:', s)

    num_batches = int(num_examples / BATCH_SIZE)
    batches_X = np.array_split(X, num_batches)
    batches_y = np.array_split(y, num_batches)
    
    for i in range(epochs):
        for xs, ys in zip(batches_X, batches_y):
            gradients       = vgradient_per_ex(theta, xs, ys)
            noisy_avg_grad  = trimmed_mean(gradients, t, m, lower, upper, sigma, s)
            theta           = theta - (1/(i+1))*noisy_avg_grad

    return theta

In [512]:
# Compute the trimmed mean
def trimmed_mean(many_xs, t, m, a, b, sigma, s):
    norms = np.linalg.norm(many_xs, ord=2, axis=1) + 1e-7
    normalized_many_xs = many_xs / norms[:, np.newaxis]
    
    #print('std, non-normalized:', len(np.std(many_xs, axis=0)))
    #print('std, normalized:', len(np.std(normalized_many_xs, axis=0)))
    
#     for s1, s2 in zip(np.std(many_xs, axis=0), np.std(normalized_many_xs, axis=0)):
#         print(s1/ s2)
  
    normalized_many_xs = np.sign(many_xs)
    values, counts = np.unique(normalized_many_xs, axis=0, return_counts=True)
    ind = np.argmax(counts)
    signs = normalized_many_xs[ind]
    signs2 = np.sign(np.mean(many_xs, axis=0))
    #print(signs == signs2)
    #print(signs)
    return signs2
    1/0
    for i in range(104):
        print(np.sort(normalized_many_xs[:,i]))

    clipped_xs = np.sort(normalized_many_xs.clip(a, b), axis=0)
    n = clipped_xs.shape[0]
    trimmed_xs = clipped_xs[m:n-m]
    width = clipped_xs.shape[1]
    
    many_xs_torch = torch.from_numpy(clipped_xs)

    sens = compute_sens_torch(many_xs_torch.T, t, m, a, b, 'cpu')
    print('sens:', np.linalg.norm(sens, ord=2))
    noise = torch.tensor(lln(sigma, width))
    return np.mean(trimmed_xs, axis=0) + ((sens/s)*noise).numpy()


In [513]:
def opt_exp(eps, t, sigma):
    return 5 * (eps / t) * sigma**3 - 5 * sigma**2 - 1

def optimize_sigma(target_rho, t):
    target_eps = np.sqrt(2*target_rho)
    sigma_lower = t / target_eps
    sigma_upper = max(2*t / target_eps, 1/2)
    
    loss = opt_exp(target_eps, t, np.mean([sigma_lower, sigma_upper]))
    while np.abs(loss) > 0.001:
        #print('loss:', loss)
        if loss < 0:
            sigma_lower = np.mean([sigma_lower, sigma_upper])
        else:
            sigma_upper = np.mean([sigma_lower, sigma_upper])

        loss = opt_exp(target_eps, t, np.mean([sigma_lower, sigma_upper]))

    sigma = np.mean([sigma_lower, sigma_upper])
    s = np.exp(-(3/2) * sigma**2) * (target_eps - (t / sigma))

    return sigma, s

In [514]:
%%time
accuracy(smooth_dp_gradient_descent(20, 10))

target per-weight rho: 0.004807692307692308
sigma: 10.217568047844624 s: 1.8329446102499663e-72


/home/jnear/anaconda3/envs/torch/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: overflow encountered in exp
  This is separate from the ipykernel package so we can avoid doing imports until


CPU times: user 21.5 s, sys: 1.32 s, total: 22.8 s
Wall time: 5.82 s


0.8285050862450243

In [364]:
zcdp_eps(0.001, 1e-5)

0.21559660262893474